In [1]:
import warnings
warnings.filterwarnings(
    "ignore",
    message=".*pin_memory.*no accelerator.*"
)


#### A Notebook For Threat Hunting Using an Ensemble ML Model
Part of the DUNE project (https://github.com/opendr-io/dune) and useful for hunting threats that are resistant to conventional detection. This notebook does ensemble model anomaly detection using fields of your choice. The examples below are for analysis of CloudTrail logs. The notebook will run fourteen models by default; there are four more that can optionally be run which tend to take more time. The notebook will score each row in the dataframe according to how many models evaluated it as an outlier. The scored anomalous events can be queried, sifted, aggregated, and sorted using the tools at the end. The notebook uses the pyod package which is on Github at: https://github.com/yzhao062/pyod and has docs here: https://pyod.readthedocs.io/en/latest/pyod.models.html

In [2]:
import sys, site, os 
sys.executable, sys.version, site.getsitepackages(), os.getcwd()

('c:\\Users\\flynn\\.conda\\envs\\dune\\python.exe',
 '3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]',
 ['c:\\Users\\flynn\\.conda\\envs\\dune',
  'c:\\Users\\flynn\\.conda\\envs\\dune\\Lib\\site-packages'],
 'z:\\Desktop\\Feb Mark Three Notebooks')

In [3]:
import datetime
import pandas as pd
import pyod
import numpy as np
from numpy import percentile
import os
from random import randrange
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import sys
import time
import torch

In [ ]:
from pyod.models.cblof import CBLOF
from pyod.models.cd import CD
from pyod.models.copod import COPOD
from pyod.models.dif import DIF
from pyod.models.ecod import ECOD
from pyod.models.gmm import GMM
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.inne import INNE
from pyod.models.kde import KDE
from pyod.models.knn import KNN
from pyod.models.lmdd import LMDD
from pyod.models.loda import LODA
from pyod.models.lof import LOF
from pyod.models.lunar import LUNAR
from pyod.models.pca import PCA
from pyod.models.ocsvm import OCSVM 
from pyod.models.qmcd import QMCD
from pyod.models.rod import ROD
from pyod.models.sod import SOD

In [ ]:
#variables
pyod_out_frac=0.001
random_state_number = 42
rs = np.random.RandomState(random_state_number)  #random state

In [ ]:
!dir

In [ ]:
# for csv loading
# pyod_file_path = 'cloudtrail-medium.csv'#

In [ ]:
# for parquet loading
pyod_input = pd.read_parquet("cloudtrail-all.parquet")
pyod_raw = pyod_input.copy()
print('raw data shape and column names:')
print(pyod_input.shape)
print()
print('Choose your field names:')
pyod_input.columns

In [ ]:
# Select fields to encode
pyod_fields_to_encode = ['sourceIPAddress', 'eventSource',  'eventName', 'userAgent', 'userIdentity.arn'] 


In [ ]:
# Select the columns you want to use for outlier detection
pyod_columns_for_outliers = ['sourceIPAddress', 'eventSource',  'eventName', 'userAgent', 'userIdentity.arn'] 


In [ ]:
# dictionary of classifiers
# defines a dictionary of anomaly detection models with configurations
# using various algorithms from libraries likely associated with
# PyOD (Python Outlier Detection).
clf = { 
    'CBLOF': CBLOF(contamination=pyod_out_frac, check_estimator=False, random_state=rs),
    'IForest': IForest(contamination=pyod_out_frac,random_state=rs),
    'KNN': KNN(contamination=pyod_out_frac),
    'AvKNN': KNN(method='mean', contamination=pyod_out_frac),
    'LOF':LOF(n_neighbors=35, contamination=pyod_out_frac),
     ### added new ####
    'ROD':ROD(contamination=pyod_out_frac),
    'CD' : CD(contamination=pyod_out_frac),
    'COPOD' : COPOD(contamination=pyod_out_frac),
    'ECOD' : ECOD(contamination=pyod_out_frac),
    'GMM' : GMM(contamination=pyod_out_frac),
    'HBOS' : HBOS(contamination=pyod_out_frac),
    'INNE' : INNE(contamination=pyod_out_frac),
    'LODA' : LODA(contamination=pyod_out_frac),
    #'QMCD' : QMCD(contamination=pyod_out_frac),
    # Optional classifiers; these will take longer to run
    #'OCSVM': OCSVM(contamination=pyod_out_frac), # One-class SVM 182 seconds
    #'DIF': DIF(contamination=pyod_out_frac), # 148 seconds
    #'KDE': KDE(contamination=pyod_out_frac), # 169 seconds
    #'LUNAR': LUNAR(contamination=pyod_out_frac), # 227 seconds i.e. 4 minutes
} 

In [ ]:
pyod_num_samples = pyod_input.shape[0]
pyod_fraction_of_inliers = (1. - pyod_out_frac) 
# (1 - fraction of outliers)
pyod_num_inliers = int(pyod_fraction_of_inliers * pyod_num_samples) 
# fraction of inliers * total number of samples
pyod_num_outliers = int(pyod_out_frac * pyod_num_samples) 
# fraction of outliers * total number of samples
print('No. of inliers: %i' % pyod_num_inliers)
print('No. of outliers: %i' % pyod_num_outliers)
print('dataframe shape:')
print(pyod_input.shape)

In [ ]:
# convert the data type of each column to category
# replace the original categorical values with their 
# corresponding integer codes. 
for field in pyod_fields_to_encode:
    pyod_input[field] = pyod_input[field].astype('category')
    pyod_input[field] = pyod_input[field].cat.codes

In [ ]:
PX = pyod_input.loc[:, pyod_columns_for_outliers].values
print(PX)

In [ ]:
# extract columns from a pandas DataFrame and convert them into a NumPy array. 
for i, classifier in enumerate(clf.keys()):
    print('Model', i + 1, ':' + classifier) 

In [ ]:
# iterates over a dictionary of classifiers and add a new column
# to the dataframe for each classifier to store classifications 
# initialized to zero
for i, (classifier_name, classifier) in enumerate(clf.items()):
    pyod_input[str('out_'+ classifier_name)] = 0

In [ ]:
now = datetime.datetime.now()
print(f"{str(now)}")

### Estimator: This cell will forecast runtimes for the loaded dataframe and remove any excessively expensive models according to runtime with a default limit of 60 seconds. If your row and cardinality counts are supernumerary, consider running on batches or chunks. 


In [ ]:

# Build PX from the working dataframe
PX = pyod_input.loc[:, pyod_columns_for_outliers].values
n_full = len(PX)

def time_model(model, X):
    t0 = time.perf_counter()
    model.fit(X)
    return time.perf_counter() - t0

def quick_forecast(
    model,
    X,
    sample_frac=0.10,
    sizes=(1000, 3000, 7000),
    repeats=3,
    seed=42,
):
    rng = np.random.default_rng(seed)
    n = len(X)
    sample_n = max(1000, int(n * sample_frac))
    sample_n = min(sample_n, n)
    idx_sample = rng.choice(n, size=sample_n, replace=False)
    Xs = X[idx_sample]

    results = []
    for s in sizes:
        s = min(s, len(Xs))
        if s < 10:
            continue
        times = []
        for _ in range(repeats):
            idx = rng.choice(len(Xs), size=s, replace=False)
            t = time_model(model, Xs[idx])
            times.append(t)
        t_med = float(np.median(times))
        results.append((s, t_med))

    # Enforce non-decreasing times to avoid negative exponent
    fixed = []
    last_t = 0.0
    for s, t in sorted(results):
        t = max(t, last_t)
        fixed.append((s, t))
        last_t = t

    if len(fixed) < 2:
        return 0.0, fixed, 0.0, 0.0

    # Fit power law t = a * n^b on log-log
    ns = np.array([p[0] for p in fixed], dtype=float)
    ts = np.array([p[1] for p in fixed], dtype=float)
    logn = np.log(ns)
    logt = np.log(ts + 1e-9)  # avoid log(0)
    b, loga = np.polyfit(logn, logt, 1)
    b = max(b, 0.0)  # clamp negative exponent
    a = np.exp(loga)
    t_pred = a * (n_full ** b)
    return t_pred, fixed, a, b

clf_filtered = dict(clf)
max_seconds = 60
excluded = []
predicted_times = {}

for name, model in list(clf.items()):
    t_pred, results, a, b = quick_forecast(model, PX)
    predicted_times[name] = t_pred
    print(f"{name}: samples={results} | a={a:.3e}, b={b:.3f} | predicted={t_pred:.1f}s")
    if t_pred > max_seconds:
        print(f"  -> removing {name} (>{max_seconds}s)")
        excluded.append(name)
        clf_filtered.pop(name, None)

print("Excluded:", excluded)
print("Estimation Complete")


In [ ]:
# model runner mark three - more efficient and cleaner version - tested on 300k row dataframes to run most models under 60s

t0 = time.perf_counter()
np.random.seed(42)
algo_failed = []

# Pre-create output columns once
for classifier_name in clf_filtered.keys():
    pyod_input[f'out_{classifier_name}'] = np.nan

for i, (classifier_name, classifier) in enumerate(clf_filtered.items()):
    begin = time.perf_counter()
    now = datetime.datetime.now()
    print(f"{now}: #{i + 1} fitting {classifier_name}")
    try:
        classifier.fit(PX)
        y_pred = classifier.predict(PX)
        pyod_input[f'out_{classifier_name}'] = y_pred
    except Exception as e:
        print(f"{now}: Error in classifier {classifier_name} {e}")
        algo_failed.append(classifier_name)

    sec = time.perf_counter() - begin
    now = datetime.datetime.now()
    print(f"{now}: Total runtime of {classifier_name} is {sec} seconds")

print(f"Total cell time: {time.perf_counter() - t0:.2f}s")


In [ ]:
now = datetime.datetime.now()
print('time check:')
print(f"{str(now)}")
if not algo_failed:
    print("✓ All anomaly detection algorithms ran successfully.")
else:
    print("⚠ Some algorithms failed:")
    for name, err in algo_failed.items():
        print(f"  - {name}: {err}")

In [ ]:
# filter and process results from the models
out_algos = []
for c in pyod_input.columns:
    if c.startswith('out_'):
        out_algos.append(c)

#calculating the rank which is the sum of the number of algorithms 
# which detected a row as an outlier
pyod_df_small =  pyod_input[out_algos].copy()
pyod_df_small['rank'] = pyod_df_small.iloc[:, :-1].sum(axis=1)

# analyze the distribution of values in the 'rank' column of the pyod_df_small DataFrame.
print('total number of rank values:', (pyod_df_small['rank'].nunique()))
print('breakdown of row count by rank scores:')
print(pyod_df_small['rank'].value_counts())

In [ ]:
out_rows_indices = pyod_df_small.query('rank > 0')
# create an dataframe with the output of the models

# identify all outlier-vote columns
out_algos = [c for c in pyod_input.columns if c.startswith('out_')]

# sum votes across algorithms → rank
pyod_input['rank'] = pyod_input[out_algos].sum(axis=1)
pyod_input

In [ ]:
pyod_row_indices = pyod_input.index[pyod_input['rank'] > 0].to_list()

In [ ]:
print(pyod_row_indices)

#### Statistics: Heatmap and Correlations Between Models

In [ ]:
ax = sns.heatmap(pyod_df_small[out_algos].corr(), annot=True)
# pearson correlation between different algorithms - this tells us 
# which algorithms are returning similar results

In [ ]:
pyod_df_small[out_algos].corr(method='pearson')
#This tells us which algorithms are similar i.e. detecting the same row as outlier

In [ ]:
# All model vote columns
out_algos = [c for c in pyod_input.columns if c.startswith('out_')]

# Sum of "outlier" votes per row
pyod_input['rank'] = pyod_input[out_algos].sum(axis=1)
pyod_input

In [ ]:
# scored anomalies in the encoded dataframe
pyod_anomalies_scored = pyod_input[pyod_input['rank'] > 0][['eventID', 'rank'] ].copy()

# list of GUIDs that were flagged
pyod_anomalous_eventids = pyod_anomalies_scored['eventID'].unique()

# pull original CloudTrail rows based on GUIDs
#pyod_raw = pd.read_parquet("cloudtrail-300.parquet")
pyod_output = pyod_raw[pyod_raw['eventID'].isin(pyod_anomalous_eventids)].copy()

# optional: keep rank and votes attached by merging back on eventid
pyod_output = pyod_output.merge(
    pyod_input[['eventID', 'rank'] + out_algos],
    on='eventID',
    how='left',
    suffixes=('', '_score')
)
pd.set_option('display.max_rows', 135)
pd.set_option('display.max_rows', 135)
pd.set_option('display.max_colwidth', None)
pyod_output

In [ ]:
# check that the event IDs match between the raw and scored data rows
pyod_ts_check = (
    pyod_raw[['eventID', 'eventTime']]
    .merge(
        pyod_output[['eventID', 'eventTime']],
        on='eventID',
        how='inner',
        suffixes=('_pyod_raw', '_ddf')
    )
)
pyod_ts_check['eventTime_match'] = (
    pyod_ts_check['eventTime_pyod_raw'] == pyod_ts_check['eventTime_ddf']
)
pyod_ts_check['eventTime_match'].value_counts()


In [ ]:
pyod_output

In [ ]:
out_algos = [c for c in pyod_input.columns if c.startswith('out_')]
pyod_input['rank'] = pyod_input[out_algos].sum(axis=1)
rank_lookup = pyod_input.loc[pyod_input['rank'] > 0, ['eventID', 'rank']]
pyod_output = pyod_raw.merge(
    rank_lookup,
    on='eventID',
    how='inner'
)
assert pyod_output['rank'].gt(0).all()
pyod_output

In [ ]:
pyod_output.to_csv('pyod_output.csv', index=False)

In [ ]:
from IPython.display import Audio, display
import numpy as np

# simple beep
display(Audio(data=np.sin(2*np.pi*440*np.linspace(0, 1, 44100)), rate=44100, autoplay=True))
